In [7]:
# Cell 0 - run in a terminal OR in a Jupyter cell with a leading "!" (e.g., !pip install ...)
!pip install --upgrade pip
!pip install pandas numpy scikit-learn lightgbm scipy tqdm matplotlib seaborn


In [13]:
!pip install lightgbm

In [21]:
import sys
print(sys.executable)

C:\Users\Sevanth Kumar\anaconda3\envs\tf_env\python.exe


In [25]:
import sys
!"{sys.executable}" -m pip install lightgbm


  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)


In [27]:
import lightgbm as lgb
print("LightGBM imported successfully! Version:", lgb.__version__)


LightGBM imported successfully! Version: 4.6.0


In [1]:
!git init


Reinitialized existing Git repository in C:/Users/Sevanth Kumar/.git/


In [3]:
!git config --global user.name "sevanth18"
!git config --global user.email "sevanthkumarj@gmail.com"


In [29]:
# Cell 1
import os
from pathlib import Path
import re
import gc
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy import sparse

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

# Metric: SMAPE (percent)
def smape(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    mask = denom == 0
    denom[mask] = 1.0
    res = np.abs(y_pred - y_true) / denom
    res[mask] = 0.0
    return np.mean(res) * 100.0

print("Imports OK. NumPy version:", np.__version__)


Imports OK. NumPy version: 2.0.2


In [31]:
# Cell 2
ROOT = Path.cwd()
DATA_DIR = Path(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset")
TRAIN_CSV = DATA_DIR / "train.csv"
TEST_CSV  = DATA_DIR / "test.csv"
OUT_FILE  = ROOT / "test_out.csv"

print("Working folder:", ROOT)
print("Expecting dataset at:", DATA_DIR)
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Dataset folder not found: {DATA_DIR}\nPlace train.csv and test.csv inside a folder named 'dataset' in the current working directory.")
if not TRAIN_CSV.exists() or not TEST_CSV.exists():
    raise FileNotFoundError(f"Missing train or test CSV. Found files: {list(DATA_DIR.iterdir())}")
print("train.csv and test.csv found. Continuing...")


Working folder: C:\Users\Sevanth Kumar
Expecting dataset at: C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset
train.csv and test.csv found. Continuing...


In [33]:
df =  pd.read_csv(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset\sample_test.csv")
df

,sample_id,catalog_content,image_link
0,217392,Item Name: Gift Basket Village Gourmet Meat an...,https://m.media-amazon.com/images/I/91GB1wC6Ob...
1,209156,"Item Name: NPG Dried Lotus Seeds 16 Oz, Uncook...",https://m.media-amazon.com/images/I/81VnzF1vkv...
2,262333,Item Name: Annies Homegrown Macaroni and Chees...,https://m.media-amazon.com/images/I/51aCDMHMnI...
3,295979,Item Name: Bear Creek Country Kitchens Creamy ...,https://m.media-amazon.com/images/I/71dzRyLGPi...
4,50604,Item Name: Japanese Kelp Kombu Umami Soup Stoc...,https://m.media-amazon.com/images/I/71Yu21cGwr...
...,...,...,...
95,289489,Item Name: Desert Essence Coconut Body Wash - ...,https://m.media-amazon.com/images/I/610gc5aZ-y...
96,297033,Item Name: Pacific Merchants Acaciaware 10- by...,https://m.media-amazon.com/images/I/81dLFRyIBB...
97,79518,Item Name: Near East Quinoa Rosemary Olive Oil...,https://m.media-amazon.com/images/I/61VqmUxzK4...
98,262905,Item Name: Foods Alive | Organic Zesty Italian...,https://m.media-amazon.com/images/I/71a7Yoov2Z...


In [35]:
df =  pd.read_csv(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset\sample_test_out.csv")
df

,sample_id,price
0,217392,62.080008
1,209156,17.189763
2,262333,96.501410
3,295979,5.652474
4,50604,23.794780
...,...,...
95,289489,87.458365
96,297033,94.364067
97,79518,26.917609
98,262905,17.719189


In [37]:
df =  pd.read_csv(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset\train.csv")
df

,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.890
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.120
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.970
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.340
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.490
...,...,...,...,...
74995,41424,Item Name: ICE BREAKERS Spearmint Sugar Free M...,https://m.media-amazon.com/images/I/81p9PcPsff...,10.395
74996,35537,"Item Name: Davidson's Organics, Vanilla Essenc...",https://m.media-amazon.com/images/I/51DDKoa+mb...,35.920
74997,249971,Item Name: Jolly Rancher Hard Candy - Blue Ras...,https://m.media-amazon.com/images/I/91R2XCcpUf...,50.330
74998,188322,Item Name: Nescafe Dolce Gusto Capsules - CARA...,https://m.media-amazon.com/images/I/51W40YU98+...,15.275


In [38]:
df =  pd.read_csv(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset\test.csv")
df

,sample_id,catalog_content,image_link
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,NaN
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...
...,...,...,...
74995,93616,Item Name: Good Seasons Zezty Italian Salad Dr...,https://m.media-amazon.com/images/I/51e9H27lgv...
74996,249434,"Item Name: Colombina Swirled Love Tiger Pops, ...",https://m.media-amazon.com/images/I/61IpkExmVt...
74997,162217,"Item Name: Kerns, Guava Nectar, 11.5 Fl Oz Can...",https://m.media-amazon.com/images/I/A1NMggyCLz...
74998,230487,Item Name: NY SPICE SHOP Licorice Candy - 1 Po...,https://m.media-amazon.com/images/I/81P69kEP5q...


In [40]:
# Cell 3
train = pd.read_csv(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset\train.csv")
test  = pd.read_csv(r"C:\Users\Sevanth Kumar\Desktop\ML_Project\dataset\test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("\nTrain columns:", train.columns.tolist())
print("\nFirst 3 train rows:")
display(train.head(3))

# Price summary
if 'price' in train.columns:
    print("\nPrice summary (train):")
    print(train['price'].describe())
else:
    raise KeyError("train.csv must contain a 'price' column (target).")

# Check catalog_content column
if 'catalog_content' not in train.columns:
    raise KeyError("train.csv must contain 'catalog_content' column.")
if 'catalog_content' not in test.columns:
    raise KeyError("test.csv must contain 'catalog_content' column.")


Train shape: (75000, 4)
Test shape: (75000, 3)

Train columns: ['sample_id', 'catalog_content', 'image_link', 'price']

First 3 train rows:


,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97



Price summary (train):
count    75000.000000
mean        23.647654
std         33.376932
min          0.130000
25%          6.795000
50%         14.000000
75%         28.625000
max       2796.000000
Name: price, dtype: float64


In [43]:
# Cell 4
def basic_text_features(df):
    df = df.copy()
    df['text'] = df['catalog_content'].fillna("").astype(str)
    df['text_len'] = df['text'].str.len().fillna(0).astype(int)
    df['num_words'] = df['text'].str.split().apply(lambda x: len(x) if isinstance(x, list) else 0)
    # simple IPQ/quantity heuristics (returns 1 when not found)
    qty_re = re.compile(r'(\d{1,4})\s?(?:x|×|\-|\s)?\s?(pack|packs|pcs|pieces|ct|count|pcs\.|packets)?\b', re.I)
    def extract_qty(s):
        if not isinstance(s, str) or s.strip()=="":
            return 1.0
        m = qty_re.search(s)
        if m:
            try:
                v = float(m.group(1))
                if 1 <= v < 10000:
                    return v
            except:
                pass
        # fallback small integer if present
        m2 = re.search(r'\b([2-9]|1[0-9]|20)\b', s)
        if m2:
            return float(m2.group(1))
        return 1.0
    df['qty_guess'] = df['text'].apply(extract_qty)
    return df

train = basic_text_features(train)
test  = basic_text_features(test)

print("Created features: text_len, num_words, qty_guess")
display(train[["catalog_content",'text_len','num_words','qty_guess']])


Created features: text_len, num_words, qty_guess


,catalog_content,text_len,num_words,qty_guess
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",91,18,12.0
1,"Item Name: Salerno Cookies, The Original Butte...",511,80,8.0
2,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",328,59,1.0
3,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,1318,211,11.0
4,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",155,28,12.0
...,...,...,...,...
74995,Item Name: ICE BREAKERS Spearmint Sugar Free M...,772,130,1.0
74996,"Item Name: Davidson's Organics, Vanilla Essenc...",1707,275,100.0
74997,Item Name: Jolly Rancher Hard Candy - Blue Ras...,142,23,5.0
74998,Item Name: Nescafe Dolce Gusto Capsules - CARA...,899,138,16.0


In [44]:
# Cell 5
# Ensure positive prices and log-transform target to reduce skew
train['price'] = train['price'].astype(float).clip(lower=0.01)

# Optional: winsorize extreme high prices (uncomment if desired)
# cap = train['price'].quantile(0.995)
# print("Capping prices at 99.5 percentile:", cap)
# train['price'] = train['price'].clip(upper=cap)

# train target
train['target'] = np.log1p(train['price'])

print("Price min/max after clipping:", train['price'].min(), train['price'].max())
print("Target (log1p) example:")
display(train[['price','target']].head())


Price min/max after clipping: 0.13 2796.0
Target (log1p) example:


,price,target
0,4.89,1.773256
1,13.12,2.647592
2,1.97,1.088562
3,30.34,3.444895
4,66.49,4.211979


In [47]:
# Cell 6
# TF-IDF on 'text'
all_text = pd.concat([train['text'], test['text']], axis=0).reset_index(drop=True)
tfidf = TfidfVectorizer(max_features=30000, ngram_range=(1,2), min_df=3)
print("Fitting TF-IDF ... (this may take a bit)")
tfidf.fit(all_text)

X_train_tfidf = tfidf.transform(train['text'])
X_test_tfidf  = tfidf.transform(test['text'])
print("TF-IDF shapes:", X_train_tfidf.shape, X_test_tfidf.shape)

# numeric features
num_cols = ['text_len','num_words','qty_guess']
scaler = StandardScaler()
scaler.fit(pd.concat([train[num_cols], test[num_cols]], axis=0))
X_train_num = scaler.transform(train[num_cols])
X_test_num  = scaler.transform(test[num_cols])

# combine sparse TF-IDF and dense numeric into a csr_matrix
from scipy import sparse
X_train = sparse.hstack([X_train_tfidf, sparse.csr_matrix(X_train_num)], format='csr')
X_test  = sparse.hstack([X_test_tfidf, sparse.csr_matrix(X_test_num)], format='csr')

y = train['target'].values
print("Final training feature shape:", X_train.shape)


Fitting TF-IDF ... (this may take a bit)
TF-IDF shapes: (75000, 30000) (75000, 30000)
Final training feature shape: (75000, 30003)


In [53]:
# Cell 7
from lightgbm import early_stopping, log_evaluation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(X_train.shape[0])
preds = np.zeros(X_test.shape[0])

lgb_params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 127,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 5,
    'seed': 42,
    'n_jobs': -1
}

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n=== Fold {fold+1} ===")
    X_tr = X_train[tr_idx]; y_tr = y[tr_idx]
    X_val = X_train[val_idx]; y_val = y[val_idx]
    lgb_tr = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val, reference=lgb_tr)
    model = lgb.train(
        lgb_params,
        lgb_tr,
        num_boost_round=3000,
        valid_sets=[lgb_tr, lgb_val],
        callbacks=[early_stopping(stopping_rounds=100), log_evaluation(period=200)]
)
    oof[val_idx] = model.predict(X_val, num_iteration=model.best_iteration)
    preds += model.predict(X_test, num_iteration=model.best_iteration) / kf.n_splits
    gc.collect()

# back-transform
oof_price = np.expm1(oof)
preds_price = np.expm1(preds)

cv_smape = smape(train['price'].values, oof_price)
print(f"\nCV SMAPE: {cv_smape:.4f}%")



=== Fold 1 ===
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.29756	valid_1's l2: 0.472391
[400]	training's l2: 0.215291	valid_1's l2: 0.465348
Early stopping, best iteration is:
[392]	training's l2: 0.217673	valid_1's l2: 0.464921

=== Fold 2 ===
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.300176	valid_1's l2: 0.453225
[400]	training's l2: 0.217676	valid_1's l2: 0.447859
Early stopping, best iteration is:
[380]	training's l2: 0.223743	valid_1's l2: 0.447375

=== Fold 3 ===
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.302128	valid_1's l2: 0.453621
[400]	training's l2: 0.21853	valid_1's l2: 0.447882
Early stopping, best iteration is:
[347]	training's l2: 0.236091	valid_1's l2: 0.447695

=== Fold 4 ===
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.303429	valid_1's l2: 0.442636
[400]	training's l2: 0.21909	valid_1's l2: 0.439258
Ear

In [51]:
# Cell 8
train['oof_pred'] = oof_price
train['abs_pct_err'] = np.abs(train['oof_pred'] - train['price']) / ((np.abs(train['oof_pred']) + np.abs(train['price']))/2)
print("Top 10 largest relative errors (train):")
display(train[['sample_id','price','oof_pred','abs_pct_err']].sort_values('abs_pct_err', ascending=False).head(10))

# Save submission



Top 10 largest relative errors (train):


,sample_id,price,oof_pred,abs_pct_err
32750,212063,0.130,16.880060,1.969430
1774,261551,0.130,14.952537,1.965523
22961,164741,526.575,6.599207,1.950491
9273,128524,1.990,142.642268,1.944964
59934,128897,0.980,51.781539,1.925703
53128,293085,0.500,23.699675,1.917354
1986,88438,1.595,72.445562,1.913831
5585,161976,0.830,35.794035,1.909349
28165,86758,1.680,68.881325,1.904764
66759,161975,0.890,35.558019,1.902327


In [52]:
submission = pd.DataFrame({'sample_id': test['sample_id'], 'price': preds_price})
submission['price'] = submission['price'].clip(lower=0.01)
submission.to_csv(OUT_FILE, index=False)
print(f"Submission saved to {OUT_FILE} (rows: {len(submission)})")
display(submission)

Submission saved to C:\Users\Sevanth Kumar\test_out.csv (rows: 75000)


,sample_id,price
0,100179,13.971438
1,245611,16.588849
2,146263,17.134144
3,95658,14.750703
4,36806,50.511990
...,...,...
74995,93616,7.240119
74996,249434,13.485603
74997,162217,3.485560
74998,230487,14.106051


In [28]:
# Cell 9 (run for guidance)
print("Done. Next improvement ideas (choose one):")
print("1) Use SBERT sentence embeddings instead of TF-IDF for better semantic understanding.")
print("2) Download images and extract CNN embeddings (MobileNet/EfficientNet) and add to features.")
print("3) Extract brand/product-specific tokens from text and use CatBoost for categorical handling.")
print("4) Hyperparameter tune LightGBM with Optuna optimizing CV SMAPE.")
print("\nIf you'd like, I can now produce code for any improvement above. Tell me which one and I'll give a step-by-step cell-by-cell code.")

Done. Next improvement ideas (choose one):
1) Use SBERT sentence embeddings instead of TF-IDF for better semantic understanding.
2) Download images and extract CNN embeddings (MobileNet/EfficientNet) and add to features.
3) Extract brand/product-specific tokens from text and use CatBoost for categorical handling.
4) Hyperparameter tune LightGBM with Optuna optimizing CV SMAPE.

If you'd like, I can now produce code for any improvement above. Tell me which one and I'll give a step-by-step cell-by-cell code.
